In [2]:
# input: patient records:
#         each record has the patient ID, race, female,  and the list of all visits seperated by #
#         each visit record  hase DaysToEvent, age, list of icd 9 codes and year
# output: patient visit records:
#       patientID, Race, Female, DaysToEvent, Age,year, list of icd9 codes entered at that visit,hot-code representation of Dxs 


In [3]:
import pandas as pd
import numpy as np
import time
import re
import os
from operator import add


In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 50000)


In [5]:
#filename = "HCUP_2003_to_2011_Patient_to_Visits.csv"
filename = "hcup_data\HCUP_2003_to_2011_Patient_to_Visits_withoutLine40.csv"
#outfile = "hcup_data\HCUP_2003_to_2011_Patient_visits_encoded120.csv"
outfile = "test.csv"
icd = pd.read_csv("icd_list.csv", header = 0)

In [16]:
#infile = "HCUP_2003_to_2011_Patient_to_Visits_withoutLine40.csv"
infile = "data_test.csv"
outfile = "patientDxs.csv"
start = os.getcwd()
infile_path = os.path.relpath('..//hcup_data//'+infile, start)
outfile_path = os.path.relpath('..//hcup_data//'+outfile, start)


In [17]:
def coding_120(i):
    if i[:1]=='V':
        return 0
    else:
        i = int(i[:3])
        code = np.where(( (i>=icd['min']) & (i<=icd['max'])))[0]
        if len(code)==0:
            return 119
        return code[0]+1

def coding_1K(i):
     if i[:1]=='V': 
        return 0
     else:
        return int(i[:3])
    
def one_hot(l,width):
    code = [0]*width 
    for i in l:
        x = [0]*width 
        x[i] = 1
        code = list( map(add, code, x) )
    return code 
#dxs = [0] *118
    #dxs[code[0]] = 1 
    #x[v][4:] = Dxs_list[1:-1]  // DXs encoded   

In [20]:
def ParsePatientsFile( infile, outfile, bunchsize = 1000):
    bunch = []
    counter = 0
    file_length = 0
    tic = time.perf_counter()
    with open(infile, "r") as r, open(outfile, "w") as w:
        for line in r:
            #print(line)
            counter +=1
            data = line.split("|")
            code1, code2 = data[1].split(",")
            dxlist = data[2].split("#")
            l = len(dxlist)
            file_length += l
            for v in range(l):
                dx = dxlist[v].split(",")
                year = dx[-1].replace("\n", "")
                if v==0 :
                    day0 = dx[0]
                codes = (dx[2:-2])
                if len(codes)==0:
                    file_length-=1
                    continue
                codes_dx = [c[2:] for c in codes if (c[:2]=='D_') & (c[3:].isnumeric())] 
                dx_1K_hot_code = one_hot([coding_120(j) for j in codes_dx],120)
                record = ','.join((data[0],code1,code2,str(int(dx[0])-int(day0)),dx[1],year,' '.join(codes), str(dx_1K_hot_code)[1:-1])) + '\n' 
                bunch.append(record)
                print(type(bunch))
            if counter == bunchsize:
                w.writelines(bunch)
                bunch = []
                counter = 0
        w.writelines(bunch)
    toc = time.perf_counter()
    print(f"Processed 11 million patients in  {toc - tic:0.4f} seconds") 
    print("number of visit records is: ", str(file_length))


In [21]:
ParsePatientsFile(infile_path, outfile_path, 1000)


<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

KeyboardInterrupt: 

In [14]:
def check_file_lines_counter_writing_speed(infile,outfile):
    tic = time.perf_counter()
    counter = 0
    with open(outfile, 'w') as out_file, open(infile, 'r') as in_file:
            for line in in_file:
                #print(line)
                counter +=1
                out_file.write(line)
    toc = time.perf_counter()
    print(f"writing in {toc - tic:0.4f} seconds") 
    print("number of records is: ", str(counter))


In [11]:
infile = "hcup_data\HCUP_2003_to_2011_Patient_visits_encoded120.csv"
#infile = "hcup_data\HCUP_2003_to_2011_Patient_to_Visits_withoutLine40.csv"
outfile = "data_test.csv"

In [12]:
check_file_lines_counter_writing_speed(infile,outfile)

writing in 197.7028 seconds
number of records is:  27182137
